In [ ]:
import os
import cv2
import shutil
import deeplabcut
import numpy as np
import pandas as pd
import os


In [ ]:
os.getcwd()

In [ ]:
start_file  ='/home/yramakrishna/DeepLabCut/conda-environments/Codes/Ken_Test_Long.mp4'

In [ ]:
dsv = start_file#deeplabcut.DownSampleVideo(start_file, width=640, height=480)

In [ ]:
# Load the video file
cap = cv2.VideoCapture(dsv)

# Get the frame rate and total number of frames
#fps = int(video.get(cv2.CAP_PROP_FPS))
fps=10
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

#*********************************************

start_sec = 0
end_sec = 0

start_frame = int(start_sec * fps)
end_frame = int((total_frames / fps) - end_sec) * fps


In [ ]:

# Define the window name and the keys to be pressed for marking
window_name = "Monkey Face Detection"
yes_key = ord('a')
no_key = ord('s')

# Define the initial frame index and label list
frame_index = start_frame
labels = []
frame_id=[]

namev = 'FaceNoface_try.mp4'

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(namev, fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

font = cv2.FONT_HERSHEY_DUPLEX
color = (255, 255, 255) # white
fontsize = 0.45
position = (20, 20)
check = 0
tot_yes = 0

cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)


while cap.isOpened():
    # Read the next frame
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if ret:
        # Display the frame in a window
        cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
        cv2.resizeWindow(window_name, 1440, 1080)
        cv2.imshow(window_name, frame)

        # Wait for a key press
        key = cv2.waitKey(0)

        # Check if the 'y' key was pressed (monkey's face is present)
        if key == yes_key:
            check = 1 ; tot_yes += 1
            frame_id.append(frame_index)
        # Check if the 'n' key was pressed (monkey's face is not present)
        elif key == no_key:
            check = 0
        elif key == ord('q'):
            break
        
        print(check)
        labels.append([frame_index, check])
        cv2.putText(frame, str(check), position, font, fontsize, color=color)
        
        video_writer.write(frame)
               

        # Increment the frame index
        frame_index += 1
    else:
        break

labels.append([tot_yes, (frame_index-tot_yes)])
labels.append([frame_id,0])
text_file = open("text_file_MANUAL.csv", "w")
labelled = pd.DataFrame(labels, columns = ['Frame_id','Face?'])
labelled.to_csv("text_file_MANUAL.csv")
#text_file.write(labels)

# Release the video capture object and close the window
cap.release()
video_writer.release()
cv2.destroyAllWindows()



In [ ]:
frame_index

In [ ]:

df_frame = pd.read_csv('text_file_MANUAL.csv', header = 0, index_col=0)
k = df_frame.loc[len(df_frame) -1].Frame_id
l = k.strip('][').split(', ')
x = [int(i) for i in l]
x